implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.


In [1]:
def naive(p, t):
    occurrences = []
    n_align = len(t) - len(p) + 1
    n_char_comp = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            n_char_comp += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, n_align, n_char_comp

In [2]:
# test naive 1
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [3]:
from bm_preproc import BoyerMoore

def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    n_align = 0
    n_char_comp = 0
    while i < len(t) - len(p) + 1:
        n_align += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            n_char_comp += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, n_align, n_char_comp

In [4]:
# test b-m 1
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [5]:
# test b-m 2
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


Question 1

How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [6]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [7]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
occurrences, num_alignments, num_character_comparisons = naive(p, genome)
print(num_alignments)

799954


Question 2

How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [8]:
print(num_character_comparisons)

984143


Question 3

How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [9]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
p_bm = BoyerMoore(p, "ACGT")
occurrences, num_alignments, num_character_comparisons = boyer_moore(p, p_bm, genome)
print(num_alignments)

127974


Question 4

Index-assisted approximate matching. In practicals, we built a Python class called Index

implementing an ordered-list version of the k-mer index.  The Index class is copied below.

In [10]:
import bisect
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index. 

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.

In [11]:
k_ind = Index(genome, 8)
print(k_ind.query("GGCGCGGT"))  # test run

[56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061]


In [12]:
def approximate_match_index(p, k_ind, t, n):  # pattern, index, text, max n of mismatches
    # pigeonhole principle: at least one of the segments should match exactly(n mismatch, but 1 more segment)
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()  # set: removes duplicates -> no worry about same match being counted twice
    index_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p)) # make sure to not run past segment length by resizing last partition
        # Extend matching segments to see if whole p matches
        matches = k_ind.query(p[start:end])
        index_hits += len(matches)
        for m in matches:
            # make sure that p doesn't run of beginning/end of t
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            # check that no more than n mismatches in rest of seq
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start) # beginning of the alignment of the entire pattern p
    return list(all_matches), index_hits

# compare to naive 2mm
def naive_2mm(p,t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        mismatch = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatch += 1
        if mismatch <= 2:
                occurrences.append(i)  # all chars matched; record
    return occurrences

In [13]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
matched_alus, n_index_hits = approximate_match_index(p, k_ind, genome, 2)
print(len(matched_alus))
matched_alus_naive2mm = naive_2mm(p, genome)
print(len(matched_alus_naive2mm))

19
19


Question 5

Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?

  (Don't consider reverse complements.)

Hint: You should be able to use the boyer_moore (or the slower naive function) to double-check your answer.


In [14]:
print(n_index_hits)

90


Question 6

Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA (first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222, and splitting into two subsequences of every other character could be represented as 121212

The following class SubseqIndex is a more general implementation of Index that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [15]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p, and only for this first subseq """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries by looping over all index keys
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [16]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

p = 'TTATAT'
print(ind.query(p[0:]))

print(ind.query(p[1:]))

[('AAA', 0), ('TTT', 1)]
[]
[1]


Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)


In [20]:
def subseq_approx_matching(p,t,kmerlen, ival, n):  # pattern, text, kmer-size of index, ival, max n of mismatches
    k_subseq_ind = SubseqIndex(t, kmerlen, ival)
    # pigeonhole principle: at least one of the segments should match exactly(n mismatch, but 1 more segment)
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()  # set: removes duplicates -> no worry about same match being counted twice
    index_hits = 0
    for i in range(n+1):
        start = i
        # make sure to not run past segment length by resizing last partition -> take minimum
        # each subseq spans certain length of p, depending on ival
        # e.g. for p=24 and ival 3: subseq 3 ends at 24, 2 at 23 and 1 at 22
        span = 1 + ival * (kmerlen - 1)
        # Extend matching segments to see if whole p matches
        matches = k_subseq_ind.query(p[start:])
        index_hits += len(matches)
        for m in matches:
            # make sure that alignment of p doesn't run of beginning/end of t
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            # check that no more than n mismatches in rest of seq
            for j in range(0, len(p)):
                if p[j] in t[m-start:m+span:ival]:
                    continue
                elif not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start) # beginning of the alignment of the entire pattern p
    return list(all_matches), index_hits


In [21]:
"""
Test Run
"""
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
pwrong = 'to-moroo  and to-morrow '
occurrences, num_index_hits = subseq_approx_matching(p, t, 8, 3, 2)
print(num_index_hits)
print(occurrences)

occurrences, num_index_hits = subseq_approx_matching(pwrong, t, 8, 3, 2)
print(num_index_hits)
print(occurrences)

6
[0, 14]
2
[0, 14]


In [28]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
occurrences, num_index_hits = subseq_approx_matching(p, genome, 8, 3, 2)
print(num_index_hits)
print(occurrences)
print(len(occurrences))

79
[712449, 651523, 558456, 273669, 349189, 681737, 717706, 658702, 551827, 83863, 579737, 262042, 635931, 471966, 262174, 67486, 84641, 596898, 160162, 84775, 100011, 322735, 479030, 199607, 472634, 489019, 454332, 719418, 724927, 719557, 108110, 707151, 766419, 657496, 160729, 56922, 23003, 191452, 704731, 551134, 747359, 421221, 147558, 364263, 137574, 421354, 465647, 783346, 429299, 141046, 124024, 746620, 22397]
53
